In [4]:
%pip install -q trl==0.8.3 transformers==4.45.2 datasets==2.21.0 peft==0.5.0 accelerate==0.28.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import transformers
import trl
import os
import datasets
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
reward_model_name = "distilbert/distilbert-base-multilingual-cased"
reward_model_path = "./models/toxic_reward"

dataset_name = "AlexSham/Toxic_Russian_Comments"

model_name = "MTSAIR/Cotype-Nano"

In [3]:
data_original = datasets.load_dataset(dataset_name, split='train')
main_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [4]:
reward_tokenizer = transformers.AutoTokenizer.from_pretrained(reward_model_name)

In [5]:
def prompt_to_chat(prompt):
    messages = [
        #{"role": "system", "content": "Ты веселый собеседник и общаешься в чате."},
        {"role": "user", "content": prompt}
    ]
    text = main_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

In [3]:

ref_model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device)

In [5]:
inputs = main_tokenizer(prompt_to_chat("Привет."), return_tensors='pt').to(device)
generated_ids = ref_model.generate(**inputs, max_new_tokens=50, do_sample=True)
print("\nGenerated text:", main_tokenizer.decode(generated_ids.flatten().cpu().numpy().tolist()))

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)



Generated text: <|im_start|>user
Привет.<|im_end|>
<|im_start|>assistant
Здравствуйте! Как я могу вам помочь сегодня?<|im_end|>


In [6]:
# To train a reward model, you need a dataset (or generator) of positive-negative pairs.
# Each training sample should be a dict with 4 keys:
#  - input_ids_chosen, attention_mask_chosen = tokenizer("A sentence that human labeler likes more")
#  - input_ids_rejected, attention_mask_rejected = tokenizer("A sentence that human labeler likes less")



class PairwiseDataset(torch.utils.data.Dataset):
    """ A dataset of all possible pairs of chosen and texts in TRT reward training format """
    def __init__(self, data, tokenizer, accepted_label: int, max_number = None, label = 'label', text = 'text'):
        super().__init__()
        self.tokenizer = tokenizer
        
        self.chosen_texts = [row[text] for row in data if row[label] == accepted_label]
        self.rejected_texts = [row[text] for row in data if row[label] != accepted_label]
        if max_number is not None:
            self.chosen_texts = self.chosen_texts[:max_number]
            self.rejected_texts = self.rejected_texts[:max_number]
        self.column_names = ["input_ids_chosen","input_ids_rejected"]
        assert self.chosen_texts, f"no texts with label {accepted_label}"
        print(f"Found {len(self.chosen_texts)} chosen and {len(self.rejected_texts)} rejected texts, {len(self)} pairs")

    def __len__(self):
        return len(self.chosen_texts) * len(self.rejected_texts)  # all pairs

    def __getitem__(self, index: int):
        chosen = self.tokenizer(self.chosen_texts[index // len(self.chosen_texts)], truncation=True)
        rejected = self.tokenizer(self.rejected_texts[index % len(self.chosen_texts)], truncation=True)
        return dict(input_ids_chosen=chosen['input_ids'], attention_mask_chosen=chosen['attention_mask'],
                    input_ids_rejected=rejected['input_ids'], attention_mask_rejected=rejected['attention_mask'])

In [8]:
TARGET_LABEL = 1   # toxic
reward_data = PairwiseDataset(data_original, reward_tokenizer, accepted_label=TARGET_LABEL, max_number=12500)

NameError: name 'PairwiseDataset' is not defined

In [9]:
sample = reward_data[31337]
print('CHOSEN:', reward_tokenizer.decode(sample['input_ids_chosen']))
print('REJECTED:', reward_tokenizer.decode(sample['input_ids_rejected']))

CHOSEN: [CLS] на хуй, безликая [SEP]
REJECTED: [CLS] примите соболезнования от всей нашей семьи. [SEP]


In [6]:

def train_reward(reward_model, reward_tokenizer, reward_data):
    training_args = trl.RewardConfig(  # like transformers.TrainingArguments
        output_dir="reward_model",
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        learning_rate=1.41e-5,
        max_steps=1_000,              # note: training may need more than 1k steps
        logging_steps=50,
        gradient_checkpointing=True,  # reduce memory usage but train ~30% slower
        gradient_checkpointing_kwargs={"use_reentrant": False},
        fp16=True,                     # disable this on CPU or on very old GPUs
        # you may add any other hyperparameters that you found useful in weeks 5-7
        #report_to="none"
    )
    
    trainer = trl.RewardTrainer(
        model=reward_model,
        args=training_args,
        tokenizer=reward_tokenizer,
        train_dataset=reward_data,
        peft_config=None,  # optionally, you may tune with LoRA, prompt-tuning, etc
    )
    
    trainer.train()
    trainer.save_model(reward_model_path)


if not os.path.isdir(reward_model_path):
    reward_model = transformers.AutoModelForSequenceClassification.from_pretrained(reward_model_name, device_map=device)
    train_reward(reward_model, reward_tokenizer, reward_data)
else:
    print("loading pretrained reward_model")
    reward_model = transformers.AutoModelForSequenceClassification.from_pretrained(reward_model_path, device_map=device)

#c533837f4fb333a7a6371e4e3073ca3bafa0e9b3

loading pretrained reward_model


In [7]:
reward_model.gradient_checkpointing_disable()
reward_model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [8]:

for sample_index in 45, 16000:
  print('TEXT:', data_original[sample_index]['text'])
  inputs = reward_tokenizer(data_original[sample_index]['text'], truncation=True, return_tensors='pt').to(device)
  with torch.no_grad():
    reward = reward_model(**inputs).logits[0, 0].item()
    print("REWARD:", reward)
  print('LABEL:', data_original[sample_index]['label'])
  print()

# note: your reward model may produce different absolute rewards.
# This is fine as long as the rewards are ordered correctly (most of the time)

TEXT: хочу чтобы у моей дочери ольги была своя квартира в новом доме🎈🎈🎈🌻🌻🌼😇💖
REWARD: -4.1432342529296875
LABEL: 0

TEXT: хабаровск город герой,а в одноклассниках сидят дебилы и упоротые путинцы.
REWARD: 4.705625534057617
LABEL: 1



In [13]:
data_original_test = datasets.load_dataset(dataset_name, split='test')
reward_test_data = PairwiseDataset(data_original_test, reward_tokenizer, accepted_label=TARGET_LABEL)

Found 4460 chosen and 20369 rejected texts, 90845740 pairs


In [14]:

def get_freq(data, name, max_len = None):
    total = len(data)
    if max_len is not None and max_len < total:
        total = max_len
    run_total = 0
    run_ok = 0
    for idx in range(total):
      sample = data[idx]
      chosen = {"input_ids" : torch.LongTensor([sample["input_ids_chosen"]]).to(device), "attention_mask" : torch.LongTensor([sample["attention_mask_chosen"]]).to(device)}
      rejected = {"input_ids" : torch.LongTensor([sample["input_ids_rejected"]]).to(device), "attention_mask" : torch.LongTensor([sample["attention_mask_rejected"]]).to(device)}
      with torch.no_grad():
        chosen_reward = reward_model(**chosen).logits[0, 0].item()
        rejected_reward = reward_model(**rejected).logits[0, 0].item()
        if chosen_reward > rejected_reward:
            run_ok += 1
      run_total += 1  
      if run_total % 100 == 0:
          proc = run_total/total * 100
          freq = run_ok / run_total * 100
          print(f'Proccessed {name}: {proc:5.3f} freq:{freq:5.3f}')
get_freq(reward_data, "train",1000)
get_freq(reward_test_data, "test",1000)

Proccessed train: 10.000 freq:99.000
Proccessed train: 20.000 freq:99.500
Proccessed train: 30.000 freq:99.667
Proccessed train: 40.000 freq:99.750
Proccessed train: 50.000 freq:99.800
Proccessed train: 60.000 freq:99.833
Proccessed train: 70.000 freq:99.857
Proccessed train: 80.000 freq:99.875
Proccessed train: 90.000 freq:99.889
Proccessed train: 100.000 freq:99.900
Proccessed test: 10.000 freq:100.000
Proccessed test: 20.000 freq:100.000
Proccessed test: 30.000 freq:100.000
Proccessed test: 40.000 freq:100.000
Proccessed test: 50.000 freq:100.000
Proccessed test: 60.000 freq:100.000
Proccessed test: 70.000 freq:100.000
Proccessed test: 80.000 freq:99.875
Proccessed test: 90.000 freq:99.889
Proccessed test: 100.000 freq:99.900


In [15]:
def generate(inputs):
    return ref_model.generate(**inputs, 
                              min_length=-1, 
                              max_new_tokens=128, 
                              do_sample=True, 
                              top_k=0, 
                              top_p=1.0, 
                              pad_token_id=main_tokenizer.eos_token_id)
                              
                              
                     #         max_new_tokens=50, 
                      #              temperature=0.8,  # Increased from 0.7
                       #            top_k=50,         # Added top_k sampling
                        #           top_p=0.95,       # Added nucleus sampling
                         #          do_sample=True ,   # Enable sampling
                          #         repetition_penalty=1.05)

inputs = main_tokenizer(["Это было"] * 5, return_tensors='pt').to(device)
cands = generate(inputs)
for candidate in cands:
  print(main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist()))

Это было бы замечательно, но у меня нет времени на такие разговоры. Пожалуйста, не беспокойтесь об этом. Спасибо за ваше время и помощь! Желаю вам всего самого хорошего! Удачи в работе и в жизни. 

---

Спасибо за ваш ответ. Я понимаю, что у вас бывает много работы и обязанностей. Надеюсь, что ваши дела будут успешно завершены, и вы сможете расслабиться после трудового дня. Если у вас возникнут какие-либо вопросы или нужна будет
Это было бы замечательно, если бы мы могли провести этот проект вместе. Но у меня нет возможности для совместной работы из-за личных обстоятельств. Я понимаю и ценю ваше время и помощь. Спасибо за ваше сотрудничество.

---

**Ответ на письмо:**

Уважаемый(ая) [Имя],

Благодарю Вас за Ваше письмо и за то, что Вы поделились своими планами по проекту. Мне очень приятно слышать о Вашем желании работ
Это было бы замечательно, если бы мы могли провести время вместе. Но у меня сейчас очень стрессовые дела, и я не могу позволить себе отвлекаться на встречи или общение 

In [16]:
import numpy as np
def infer(prompt, N = 16):
    inputs = main_tokenizer([prompt] * 16, return_tensors='pt').to(device)
    cands = generate(inputs);
    texts = []
    rewards = []
    for candidate in cands:
        cand_text = main_tokenizer.decode(candidate.flatten().cpu().numpy().tolist(), skip_special_tokens=True)
        texts.append(cand_text)

    for text in texts:
        inputs = reward_tokenizer(text, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            reward = reward_model(**inputs).logits[0, 0].item()
        rewards.append(reward)
    return texts[np.argmax(rewards)]

neutral_prompts = ["Этот фильм", "Я пошел гулять", "И что"]

for p in neutral_prompts:
    res = infer(p).replace('<br />', '\n')
    print('='*20)
    print(res)

Этот фильм был снят в 1950 году и является классикой кинематографа. Он получил множество наград за свои уникальные спецэффекты, которые были на тот момент новаторскими. В фильме рассказывается история молодого человека, который сталкивается с различными трудностями и испытаниями в своей жизни. Однако, несмотря на все препятствия, он всегда сохраняет оптимизм и верит в лучшее будущее.

Известно, что режиссер этого фильма — Эрнест Хоппер
Я пошел гулять в парк и встретил кота. Кот был очень милый, его глаза светились радостью. Я решил оставить ему еду и ушел. Возвращаясь домой, я заметил, что на дороге лежал мячик. Не зная, куда его положить, я решился взять его с собой. Как мне быть с этим мячиком? 

Как вы думаете, какое решение будет лучше: отдать мячик кому-то, кто сможет его принести домой или оставить его здесь, чтобы он мог найти хозяина?
И что, если я начну с того, что у меня есть 100 единиц вложенных в акции компании. Я хочу узнать, какую сумму мне нужно будет вывести из этих акц

In [9]:
# Note: this code is specific to IMDB; you will need to re-write it for other tasks
data_for_rlhf = data_original.filter(lambda row: len(row['text']) > 200, batched=False)
data_for_rlhf = data_for_rlhf.remove_columns(['label'])
sample_length = trl.core.LengthSampler(2, 8)  # use the first 2-8 tokens as query

def select_query_and_tokenize(sample):
    query_ids = main_tokenizer.encode(sample["text"])[: sample_length()]
    sample["query"] = main_tokenizer.decode(query_ids)  # query is the only required column
    sample["input_ids"] = query_ids  # to avoid re-tokenizing later
    return sample  # we do not need the rest - it will be generated by the model

data_for_rlhf = data_for_rlhf.map(select_query_and_tokenize, batched=False)
data_for_rlhf.set_format(type="torch")

In [10]:
from typing import List
def compute_reward(texts: List[str]) -> torch.Tensor:
  inputs = reward_tokenizer(texts, truncation=True, padding=True, return_tensors='pt').to(device)
  with torch.no_grad():
    return reward_model(**inputs).logits[:, 0]

In [11]:
compute_reward([data_original[45]['text'], data_original[16000]['text']])  # test on human-written reviews

tensor([-4.1432,  4.7056], device='cuda:0')

Finally, we move to RL training. In this tutorial, we'll train LoRA adapters and not the full model.

In [12]:
import peft
#peft_config = peft.LoraConfig(
#    task_type=peft.TaskType.CAUSAL_LM, r=32, lora_alpha=32, lora_dropout=0.0, inference_mode=False
#)
# Настройка квантизации
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
# Настройка LoRA
peft_config = peft.LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "mlp.down_proj",
        "self_attn.k_proj",
        "self_attn.o_proj",
        "mlp.up_proj",
        "self_attn.v_proj",
        "mlp.gate_proj",
        "self_attn.q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=peft.TaskType.CAUSAL_LM
)
# reload main model as AutoModelForCausalLMWithValueHead - with an extra head needed for PPO
main_tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
main_tokenizer.pad_token = main_tokenizer.eos_token

main_model = trl.AutoModelForCausalLMWithValueHead.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)
main_model.generation_config = transformers.GenerationConfig()
main_model = peft.get_peft_model(main_model, peft_config, adapter_name='default')
main_model.print_trainable_parameters()

trainable params: 18,464,768 || all params: 1,562,180,609 || trainable%: 1.1819867621977376


In [13]:
training_args = trl.PPOConfig(
    model_name=main_model.config._name_or_path,
    gradient_accumulation_steps=4,
    learning_rate=1.41e-4,
    batch_size=16,
    ppo_epochs=4,                 # PPO performs this many updates per training batch
    mini_batch_size = 4,
    
)

ppo_trainer = trl.PPOTrainer(
    training_args, model=main_model.model, tokenizer=main_tokenizer,
    dataset=data_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0]),
    
)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

#training_args = trl.PPOConfig(
#    output_dir = "./models/ppotrain",
#    gradient_accumulation_steps=1,
#    learning_rate=1.41e-5,
#    batch_size=64,
#    num_ppo_epochs =4,                 # PPO performs this many updates per training batch
#)

#ppo_trainer = trl.PPOTrainer(
#    training_args, model=main_model.model, tokenizer=main_tokenizer,
#    train_dataset=data_for_rlhf, data_collator=lambda data: dict((key, [d[key] for d in data]) for key in data[0]),
#    reward_model = reward_model,
#    ref_model = ref_model
#)  # note: we pass main_model.model because PPOTrainer checks for one of several supported model types ...
# ... main_model.model is a model with adapters, which is supported. main_model itself is a wrapper that is not supported

In [14]:
from tqdm.auto import tqdm
max_steps = 400   # can be insufficient for some tasks - watch your learning curves
generation_kwargs = dict(
    min_length=-1, max_new_tokens=128, do_sample=True, top_k=0, top_p=0.8, temperature=0.4, repetition_penalty=1.05, pad_token_id=main_tokenizer.eos_token_id)
#                                  ^-- task-specific parameter!
with tqdm(enumerate(ppo_trainer.dataloader), total=max_steps) as progressbar:
  # note: ppo_trainer.dataloader is just a regular dataloader of queries, no RL-specific magic :)
  for epoch, batch in progressbar:
    if epoch >= max_steps:
        break

    # Rollout stage: generate continuations from batch queries using main_model
    response_tensors = ppo_trainer.generate(batch['input_ids'], **generation_kwargs)
    # ^-- list of tensors of token ids from main model tokenizer

    # de-tokenize responses to strings (since reward model uses a different tokenizer)
    batch["response"] = [main_tokenizer.decode(response.squeeze()) for response in response_tensors]
    # note: response_tensors already contain query tokens, so we don't need to add queries manually.
    # This may not be true for other tasks: check this manually by viewing batch["response"] and batch["query"]


    # Evaluation stage
    rewards = compute_reward(batch['response'])

    # Update stage
    stats = ppo_trainer.step(batch['input_ids'], response_tensors, list(rewards.split(1)))
    stats['rewards/mean'] = rewards.mean().item()

    print("-" * 30, 'STEP', epoch, '-' * 30)
    print(f'rewards/mean:\t{stats["rewards/mean"]:.9f}\t<---- average reward over this batch (higher=better, noisy)')
    print(f'ppo/returns/mean:\t{stats["ppo/returns/mean"]:.9f}\t<---- model-estimated average discounted reward')
    print(f'objective/kl:\t{stats["objective/kl"]:.9f}\t<---- how far we are from the original model (regularizer)')
    print()

    ppo_trainer.log_stats(stats, batch, list(rewards.split(1)))

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
  0%|          | 1/400 [00:36<4:00:25, 36.15s/it]

------------------------------ STEP 0 ------------------------------
rewards/mean:	-4.175922394	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	-1.166327119	<---- model-estimated average discounted reward
objective/kl:	0.000000000	<---- how far we are from the original model (regularizer)



  0%|          | 2/400 [01:12<3:59:00, 36.03s/it]

------------------------------ STEP 1 ------------------------------
rewards/mean:	-3.899804592	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	-1.071911097	<---- model-estimated average discounted reward
objective/kl:	0.872269511	<---- how far we are from the original model (regularizer)



  1%|          | 3/400 [01:48<3:58:06, 35.99s/it]

...

------------------------------ STEP 87 ------------------------------
rewards/mean:	-3.441480637	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	-3.232239485	<---- model-estimated average discounted reward
objective/kl:	-4.386944771	<---- how far we are from the original model (regularizer)



/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -1.50 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(
 22%|██▏       | 89/400 [53:01<3:08:21, 36.34s/it]

------------------------------ STEP 88 ------------------------------
rewards/mean:	-4.058603764	<---- average reward over this batch (higher=better, noisy)
ppo/returns/mean:	-3.444664955	<---- model-estimated average discounted reward
objective/kl:	-1.498209238	<---- how far we are from the original model (regularizer)



 22%|██▏       | 89/400 [53:27<3:06:48, 36.04s/it]


KeyboardInterrupt: 

In [28]:

text= prompt_to_chat("пошел ты в жопу")
model_inputs = main_tokenizer([text], return_tensors="pt")#.to(main_model.device)

generated_ids = main_model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = main_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Ваш вопрос содержит ненормативную лексику, которая может быть воспринята как оскорбительная. Если ваш запрос имеет другой контекст или цель, пожалуйста, уточните его, чтобы я мог предоставить более точный ответ.

Если же это была попытка задать какой-то вопрос на русском языке, который несет в себе негативный смысл (например, "ты куда пошел?" с намёком на некий негативный опыт), то стоит отметить, что такие вопросы могут быть неуместны в дискуссии и лучше промолчать.

Для того чтобы получить полезный ответ, пожалуйста, переформулируйте свой вопрос, избегая неприемлемых выражений и контекстов.
